## PID Control:

### A simulation of a PID control feedback loop 

In [26]:
# Import relevant packages

import numpy as np
from bokeh.io import show, output_notebook
from bokeh.plotting import figure 
from bokeh.layouts import row
output_notebook()

Loading BokehJS ...

In [27]:
# Define simulation parameters

timesteps = 120
samplingtime = 0.2 
maxsetpoint = 10.0
timepoints = []
outputs = []
setpoints = []
errors = []

timepoints = np.zeros(timesteps)
outputs = np.zeros(timesteps)
setpoints = np.zeros(timesteps)
errors = np.zeros(timesteps)

In [28]:
# Define the class for computing the PID feedback

class PID:
    # Function to read input PID parameters
    def __init__(self, P, I, D):
        self.kp = P
        self.ki = I
        self.kd = D

    # Function to initialize variables
    def initialize(self):
        global timepoints 
        timepoints.fill(np.nan)
        global outputs 
        outputs.fill(np.nan)
        global setpoints 
        setpoints.fill(np.nan)
        global errors 
        errors.fill(np.nan)
        self.processval = 0.0  # Process value
        self.controlvar = 0.0  # Control variable
        self.setpoint = 0.0  # Setpoint
        self.error = 0.0
        self.prop_term = 0.0
        self.inte_term = 0.0
        self.deri_term = 0.0
        self.lasterr = 0.0
        self.currenttime = time.time()
        self.lasttime = self.currenttime
    
    # Function for the PID control logic
    def controlLoop(self):
        self.error = self.setpoint - self.processval
        self.prop_term = self.kp * self.error
        self.inte_term = self.inte_term + self.ki * self.error 
        self.deri_term = self.kd * (self.error - self.lasterr) 
        self.controlvar = self.prop_term + self.inte_term + self.deri_term
        self.lasterr = self.error

#### Modify the code in the cell below to change PID parameters, chose an input function, and modify the noise level

In [29]:
# Enter PID control parameters, input functions, and noise levels

Kp = 0.1  # Proportional gain
Ki = 0.01  # Integrative gain
Kd = 0.001  # derivative gain
Noise = 0.0  # Gaussian noise: 0 = no noise, anything between 0 and 1 = Gaussian noise with that sigma
InputFunction = 'step'  # Choose from null, step, square, sine
freq = 1  # Frequency for the sine and square wave innputs: Enter a number between 1 and 10

In [30]:
# Run the PID feedback loop

# Create PID class object
pid = PID(Kp, Ki, Kd)
pid.initialize()
delay = 20
for nn in range(1, timesteps, 1):

    # Inputs
    if nn > delay:
        if Noise == 0:
            gauss_noise = 0
        else:
            gauss_noise = Noise * np.random.randn()
        if InputFunction == 'step': 
            # step function
            pid.setpoint = maxsetpoint + gauss_noise
        elif InputFunction == 'square':
            # square wave
            pid.setpoint = maxsetpoint * np.sign(np.sin(4 * freq * np.pi * (nn - delay) / timesteps)) + gauss_noise
        elif InputFunction == 'sine':
            # sine wave
            pid.setpoint = maxsetpoint * np.sin(4 * freq * np.pi * (nn - delay) / timesteps) + gauss_noise
        elif InputFunction == 'null':
            pid.setpoint = gauss_noise
            
    # Outputs 
    if nn > delay:
        pid.processval = pid.processval + (pid.controlvar - (1.0 / nn))
    pid.controlLoop()
    
    # Update arrays
    global outputs
    outputs[nn] = pid.processval
    global timepoints
    timepoints[nn] = nn 
    global setpoints
    setpoints[nn] = pid.setpoint
    global errors
    errors[nn] = pid.error
    

In [34]:
# Setup plots

# Figure 1
p1 = figure(plot_width=400, plot_height=300, 
            x_range=[0, timesteps], y_range=[-1.5 * maxsetpoint, 1.5 *maxsetpoint],
            title='PID controller input and output')
p1.line(timepoints, setpoints, line_color='cornflowerblue', legend='input',
            line_width=2)
p1.line(timepoints, outputs, line_color='indianred', legend='output',
            line_width=2)
p1.legend.location = 'bottom_left'  
# p1.toolbar.logo = None
# p1.toolbar_location = None

# Figure 2
p2 = figure(plot_width=400, plot_height=300, 
            x_range=[0, timesteps], title='PID controller error')
p2.line(timepoints, errors, line_color='indigo', legend='error',
            line_width=2)
p2.legend.location = 'bottom_left'  
# p2.toolbar.logo = None
# p2.toolbar_location = None

# Display plots
show(row(p1, p2))